The combined ground obs needs to be run, and both UNETtry2 as well as SRCNN need to run.. This code animates and plots comparisons. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation

# CONFIG: Update paths if needed
OBS_INTERP_FILE   = "obs_interp.npy"      # Save your (T, H, W) observations here
MODEL_INTERP_FILE = "model_interp.npy"    # Save your (T, H, W) TIE-GCM interpolated outputs here
EPOCHS_FILE       = "all_epochs.npy"      # Save all_epochs (datetime64 array) here
LATS_FILE         = "model_lats.npy"      # 1D lat array
LONS_FILE         = "model_lons.npy"      # 1D lon array

UNET_SR_FILE      = "superres_vtec.npy"   # (T, H, W, 1)
SRCNN_SR_FILE     = "srcnn_superres_vtec.npy" # (T, H, W, 1)

# 1. --- LOAD DATA ---
obs_interp   = np.load(OBS_INTERP_FILE)
model_interp = np.load(MODEL_INTERP_FILE)
all_epochs   = np.load(EPOCHS_FILE, allow_pickle=True)         # <-- FIX HERE
model_lats   = np.load(LATS_FILE)
model_lons   = np.load(LONS_FILE)
Y_pred_unet  = np.load(UNET_SR_FILE)
Y_pred_srcnn = np.load(SRCNN_SR_FILE)


# Make sure they're aligned in time
T = min(obs_interp.shape[0], model_interp.shape[0], Y_pred_unet.shape[0], Y_pred_srcnn.shape[0])
obs_interp   = obs_interp[:T]
model_interp = model_interp[:T]
all_epochs   = all_epochs[:T]
Y_pred_unet  = Y_pred_unet[:T, ..., 0]
Y_pred_srcnn = Y_pred_srcnn[:T, ..., 0]

# 2. --- COMPUTE DIFFERENCES ---
diff_model  = model_interp - obs_interp     # (T, H, W)
diff_unet   = Y_pred_unet  - obs_interp    # (T, H, W)
diff_srcnn  = Y_pred_srcnn - obs_interp    # (T, H, W)

# 3. --- STATISTICS/HISTOGRAMS ---
for lbl, arr in zip(['TIE-GCM', 'U-Net SR', 'SRCNN SR'], [diff_model, diff_unet, diff_srcnn]):
    print(f"{lbl} -- Difference: mean={np.nanmean(arr):.3f}, std={np.nanstd(arr):.3f}, median={np.nanmedian(arr):.3f}")

plt.figure(figsize=(10,6))
for lbl, arr in zip(['TIE-GCM', 'U-Net SR', 'SRCNN SR'], [diff_model, diff_unet, diff_srcnn]):
    plt.hist(arr.flatten()[~np.isnan(arr.flatten())], bins=60, alpha=0.5, label=lbl, density=True)
plt.xlabel('Prediction - Observation (TECU)')
plt.ylabel('Frequency')
plt.legend()
plt.title('Distribution of Difference Fields')
plt.tight_layout()
plt.savefig('diff_histogram_compare.png')
plt.close()

# 4. --- STATIC SPATIAL MAPS (select frames) ---
idxs = np.linspace(0, T-1, min(8, T)).astype(int)
for i, idx in enumerate(idxs):
    fig, axs = plt.subplots(1, 3, figsize=(20, 5), subplot_kw={'projection': ccrs.PlateCarree()})
    vlim = max(np.max(np.abs(diff_model[idx])), np.max(np.abs(diff_unet[idx])), np.max(np.abs(diff_srcnn[idx])), 1)
    vmin, vmax = -vlim, vlim
    titles = [
        f"TIE-GCM - Obs\n{str(pd.to_datetime(all_epochs[idx]))[:16]}",
        f"U-Net SR - Obs\n{str(pd.to_datetime(all_epochs[idx]))[:16]}",
        f"SRCNN SR - Obs\n{str(pd.to_datetime(all_epochs[idx]))[:16]}"
    ]
    data = [diff_model[idx], diff_unet[idx], diff_srcnn[idx]]
    for ax, dat, title in zip(axs, data, titles):
        im = ax.pcolormesh(model_lons, model_lats, dat, vmin=vmin, vmax=vmax, shading='auto', cmap='bwr')
        ax.coastlines()
        ax.set_title(title)
        cb = plt.colorbar(im, ax=ax, orientation='vertical')
        cb.set_label('Difference (TECU)')
    plt.tight_layout()
    plt.savefig(f"static_difference_{i:02d}.png")
    plt.close(fig)

# 5. --- ANIMATION ---
def make_diff_animation(diff_array, label, all_epochs, model_lats, model_lons, vmax=None):
    from matplotlib.animation import FuncAnimation
    vmax = vmax or np.nanmax(np.abs(diff_array))
    vmin = -vmax
    fig, ax = plt.subplots(figsize=(10,4), subplot_kw={'projection': ccrs.PlateCarree()})
    mesh = ax.pcolormesh(model_lons, model_lats, diff_array[0], vmin=vmin, vmax=vmax, shading='auto', cmap='bwr')
    ax.coastlines()
    cb = plt.colorbar(mesh, ax=ax, orientation='vertical')
    cb.set_label('Difference (TECU)')
    title = ax.set_title(f"{label} - Obs {str(pd.to_datetime(all_epochs[0]))[:16]}")
    def update(idx):
        mesh.set_array(diff_array[idx].ravel())
        title.set_text(f"{label} - Obs {str(pd.to_datetime(all_epochs[idx]))[:16]}")
        return mesh, title
    anim = FuncAnimation(fig, update, frames=diff_array.shape[0], blit=False)
    anim.save(f"anim_{label.lower().replace(' ', '_')}_minus_obs.gif", writer='pillow', fps=4)
    plt.close(fig)
    print(f"Animation saved: anim_{label.lower().replace(' ', '_')}_minus_obs.gif")

# Animation (per method)
make_diff_animation(diff_model, 'TIE-GCM', all_epochs, model_lats, model_lons)
make_diff_animation(diff_unet,  'U-Net SR', all_epochs, model_lats, model_lons)
make_diff_animation(diff_srcnn, 'SRCNN SR', all_epochs, model_lats, model_lons)

print("All difference plots, histograms, and animations complete.")

